In [1]:
import sys
sys.path.append('/host/d/Github/')
import nibabel as nb
import glob
import os
import numpy as np
import shutil
import Diffusion_denoising_thin_slice.functions_collection as ff


/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
save_folder = '/host/d/Data/PCCT/data/soft_thins_xy'
patient_list = ff.find_all_target_files(['*'],os.path.join('/host/d/Data/PCCT/data/soft_thins'))
for i in range(0, patient_list.shape[0]):
    patient_id = os.path.basename(patient_list[i])
    print('patinet id:', patient_id)
    ff.make_folder([os.path.join(save_folder, patient_id)])

    filename = os.path.join(patient_list[i],'soft_thins_0_noblank.nii.gz')
    img = nb.load(filename)
    affine = img.affine
    data = img.get_fdata()[:,:,30:80]
    nb.save(nb.Nifti1Image(data, affine), os.path.join(save_folder, patient_id, 'soft_thins_0_noblank_sliced.nii.gz'))
    

    # gaussian_folder = os.path.join(patient_list[i],'gaussian_random_0')
    # save_gaussian_folder = os.path.join(save_folder, patient_id, 'gaussian_random_0')
    # ff.make_folder([save_gaussian_folder])

    # shutil.copyfile(os.path.join(gaussian_folder, 'recon_all_sliced.nii.gz'), os.path.join(save_gaussian_folder, 'recon_all_sliced.nii.gz'))
    # shutil.copyfile(os.path.join(gaussian_folder, 'recon_even_sliced.nii.gz'), os.path.join(save_gaussian_folder, 'recon_even_sliced.nii.gz'))
    # shutil.copyfile(os.path.join(gaussian_folder, 'recon_odd_sliced.nii.gz'), os.path.join(save_gaussian_folder, 'recon_odd_sliced.nii.gz'))

    # poisson_folder = os.path.join(patient_list[i],'poisson_random_0')
    # save_poisson_folder = os.path.join(save_folder, patient_id, 'poisson_random_0')
    # ff.make_folder([save_poisson_folder])
    # recon_all_file = os.path.join(poisson_folder, 'recon_all.nii.gz')
    # recon_all = nb.load(recon_all_file)
    # affine = recon_all.affine
    # recon_all_data = recon_all.get_fdata()[:,:,100:200]
    # nb.save(nb.Nifti1Image(recon_all_data, affine), os.path.join(save_poisson_folder, 'recon_all_sliced.nii.gz'))
    # recon_even_file = os.path.join(poisson_folder, 'recon_even.nii.gz')
    # recon_even = nb.load(recon_even_file)
    # recon_even_data = recon_even.get_fdata()[:,:,100:200]
    # nb.save(nb.Nifti1Image(recon_even_data, affine), os.path.join(save_poisson_folder, 'recon_even_sliced.nii.gz'))
    # recon_odd_file = os.path.join(poisson_folder, 'recon_odd.nii.gz')
    # recon_odd = nb.load(recon_odd_file)
    # recon_odd_data = recon_odd.get_fdata()[:,:,100:200]
    # nb.save(nb.Nifti1Image(recon_odd_data, affine), os.path.join(save_poisson_folder, 'recon_odd_sliced.nii.gz'))
    # print(f'Finished patient {patient_id}')
    
    


patinet id: 1
patinet id: 11
patinet id: 12
patinet id: 14
patinet id: 15
patinet id: 16
patinet id: 17
patinet id: 18
patinet id: 19
patinet id: 2
patinet id: 20
patinet id: 21
patinet id: 22
patinet id: 23
patinet id: 24
patinet id: 25
patinet id: 26
patinet id: 27
patinet id: 28
patinet id: 29
patinet id: 3
patinet id: 30
patinet id: 31
patinet id: 32
patinet id: 33
patinet id: 34
patinet id: 35
patinet id: 36
patinet id: 4
patinet id: 5
patinet id: 6
patinet id: 7
patinet id: 9
